<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/docs/examples/agent/openai_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Function Calling Mistral Agent

This notebook shows you how to use our Mistral agent, powered by function calling capabilities.

## Initial Setup 

Let's start by importing some simple building blocks.  

The main thing we need is:
1. the OpenAI API (using our own `llama_index` LLM class)
2. a place to keep conversation history 
3. a definition for tools that our agent can use.

If you're opening this Notebook on colab, you will probably need to install LlamaIndex 🦙.


In [ ]:
%pip install llama-index-llms-mistralai
%pip install llama-index-embeddings-mistralai

In [ ]:
!pip install llama-index

In [1]:
import json
from typing import Sequence, List

from llama_index.llms.mistralai import MistralAI
from llama_index.core.llms import ChatMessage
from llama_index.core.tools import BaseTool, FunctionTool

import nest_asyncio

nest_asyncio.apply()

Let's define some very simple calculator tools for our agent.

In [2]:
def multiply(a: int, b: int) -> int:
    """Multiple two integers and returns the result integer"""
    return a * b


multiply_tool = FunctionTool.from_defaults(fn=multiply)

In [3]:
def add(a: int, b: int) -> int:
    """Add two integers and returns the result integer"""
    return a + b


add_tool = FunctionTool.from_defaults(fn=add)

In [4]:
llm = MistralAI(model="mistral-large-latest")

## Initialize Mistral Agent

In [5]:
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner

agent_worker = FunctionCallingAgentWorker.from_tools([multiply_tool, add_tool], llm=llm, verbose=True)
agent = AgentRunner(agent_worker)

### Chat

In [7]:
response = agent.chat("What is (121 * 3) + 42?")
print(str(response))

Added user message to memory: What is (121 * 3) + 42?
MESSAGES: [ChatMessage(role='user', content='What is (121 * 3) + 42?', name=None, tool_calls=None)]
MESSAGES: [ChatMessage(role='user', content='What is (121 * 3) + 42?', name=None, tool_calls=None), ChatMessage(role='tool', content='363', name=None, tool_calls=None)]


MistralAPIException: Status: 400. Message: {"object":"error","message":"Unexpected role RoleEnum.tool","type":"invalid_request_error","param":null,"code":null}

In [ ]:
# inspect sources
print(response.sources)

[ToolOutput(content='363', tool_name='multiply', raw_input={'args': (), 'kwargs': {'a': 121, 'b': 3}}, raw_output=363), ToolOutput(content='405', tool_name='add', raw_input={'args': (), 'kwargs': {'a': 363, 'b': 42}}, raw_output=405)]


### Async Chat

In [ ]:
response = await agent.achat("What is 121 * 3?")
print(str(response))

STARTING TURN 1
---------------

=== Calling Function ===
Calling function: multiply with args: {
  "a": 121,
  "b": 3
}
Got output: 363

STARTING TURN 2
---------------

121 multiplied by 3 is equal to 363.


### Streaming Chat
Here, every LLM response is returned as a generator. You can stream every incremental step, or only the last response.

In [ ]:
response = agent.stream_chat(
    "What is 121 * 2? Once you have the answer, use that number to write a"
    " story about a group of mice."
)

response_gen = response.response_gen

for token in response_gen:
    print(token, end="")

STARTING TURN 1
---------------

=== Calling Function ===
Calling function: multiply with args: {
  "a": 121,
  "b": 2
}
Got output: 242

STARTING TURN 2
---------------

121 multiplied by 2 is equal to 242.

Once upon a time, in a small village, there was a group of mice who lived in a cozy little burrow. The mice were known for their intelligence and resourcefulness. They had built a tight-knit community and worked together to overcome any challenges they faced.

One sunny day, as the mice were going about their daily activities, they stumbled upon a bountiful field of ripe corn. The field was filled with tall stalks of golden corn, swaying gently in the breeze. The mice couldn't believe their luck! They knew they had to gather as much corn as possible to sustain themselves through the upcoming winter.

With their tiny paws and sharp teeth, the mice began to harvest the corn. They worked tirelessly, carrying one ear of corn at a time back to their burrow. The mice were determined to 

### Async Streaming Chat

In [ ]:
response = await agent.astream_chat(
    "What is 121 + 8? Once you have the answer, use that number to write a"
    " story about a group of mice."
)

response_gen = response.response_gen

async for token in response.async_response_gen():
    print(token, end="")

STARTING TURN 1
---------------

=== Calling Function ===
Calling function: add with args: {
  "a": 121,
  "b": 8
}
Got output: 129

STARTING TURN 2
---------------

121 plus 8 is equal to 129.

Once upon a time, in a peaceful meadow, there lived a group of mice. These mice were known for their bravery and adventurous spirit. They loved exploring the meadow and discovering new places.

One sunny day, as the mice were scurrying through the tall grass, they stumbled upon a hidden treasure. It was a small, sparkling gemstone that radiated with a mesmerizing glow. The mice were amazed by its beauty and knew that it was something special.

Excitedly, the mice decided to take the gemstone back to their burrow. They carefully carried it, taking turns to ensure its safety. As they reached their cozy home, they marveled at the gemstone's brilliance. Little did they know, this gemstone held a magical power.

As the mice gathered around the gemstone, a soft, enchanting light began to emanate from

### Agent with Personality

You can specify a system prompt to give the agent additional instruction or personality.

In [ ]:
from llama_index.agent.openai import OpenAIAgent
from llama_index.llms.openai import OpenAI
from llama_index.core.prompts.system import SHAKESPEARE_WRITING_ASSISTANT

In [ ]:
llm = OpenAI(model="gpt-3.5-turbo-0613")

agent = OpenAIAgent.from_tools(
    [multiply_tool, add_tool],
    llm=llm,
    verbose=True,
    system_prompt=SHAKESPEARE_WRITING_ASSISTANT,
)

In [ ]:
response = agent.chat("Hi")
print(response)

STARTING TURN 1
---------------

Greetings, fair traveler! How may I assist thee on this fine day?


In [ ]:
response = agent.chat("Tell me a story")
print(response)

STARTING TURN 1
---------------

Of course, dear friend! Allow me to weave a tale for thee in the style of Shakespeare. 

Once upon a time, in a land far away, there lived a noble knight named Sir William. He was known throughout the kingdom for his bravery and chivalry. One fateful day, as Sir William rode through the enchanted forest, he stumbled upon a hidden glade.

In the glade, he discovered a beautiful maiden named Lady Rosalind. She was fair of face and gentle of heart, and Sir William was instantly captivated by her beauty. They spent hours conversing, sharing stories, and laughing together.

As the days turned into weeks, Sir William and Lady Rosalind's bond grew stronger. They found solace in each other's company and realized that they had fallen deeply in love. However, their love was not without obstacles.

Lady Rosalind's father, Lord Reginald, was a stern and overprotective man. He had already arranged a marriage for his daughter with a wealthy nobleman, Lord Percival. W